In [1]:
# NOTE: For this to work the imagenet-vgg-verydeep-19.mat file must be in the folder with the notebook.
# But it's large so I'm not checking it into git.
# You will also want at least one of the corresponding checkpoints as well.
# See: https://github.com/dilbrent/fast-style-transfer for info re: mat and ckpt files

from __future__ import print_function
import sys
sys.path.insert(0, 'src')
import numpy as np, pdb, os
import maininit
from utils import list_files

BATCH_SIZE = 4
DEVICE = '/gpu:0'

def doTransfer(in_path,out_path,checkpoint_dir):
    if not os.path.isdir(in_path):
        if os.path.exists(out_path) and os.path.isdir(out_path):
            out_path = \
                    os.path.join(out_path,os.path.basename(in_path))

        ffwd_to_img(in_path, out_path, checkpoint_dir,
                    device=DEVICE)
    else:
        files = list_files(in_path)
        full_in = [os.path.join(in_path,x) for x in files]
        full_out = [os.path.join(out_path,x) for x in files]
        allow_different_dimensions=True
        if allow_different_dimensions:
            maininit.ffwd_different_dimensions(full_in, full_out, checkpoint_dir, 
                    device_t=DEVICE, batch_size=BATCH_SIZE)
        else :
            maininit.ffwd(full_in, full_out, checkpoint_dir, device_t=DEVICE,
                    batch_size=BATCH_SIZE)

print('Init complete')

Init complete


In [2]:
doTransfer("in","out","../../fast-style-transfer/checkpoints/udnie.ckpt")
!ls -al out/*.jpg
print('Done')

Processing images of shape 488x338x3
INFO:tensorflow:Restoring parameters from ../../fast-style-transfer/checkpoints/udnie.ckpt
-rw-rw----. 1 root aid_sdcard_rw 28854 Jul 15 07:03 out/monacat.jpg
Done


In [3]:
%%HTML
<img width='100' src='in/monacat.jpg'/> <img width='100' src='out/monacat.jpg'/>